In [1]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .master("local[*]") \
    .getOrCreate()
# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/31 12:16:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Set up config

In [3]:
# set up temp config for trial run
model_train_date_str = "2017-03-01"
train_test_period_months = 0
oot_period_months = 0
train_test_ratio = 0.8

config = {}
config["model_train_date_str"] = model_train_date_str
config["train_test_period_months"] = train_test_period_months
config["oot_period_months"] =  oot_period_months
config["model_train_date"] =  datetime.strptime(model_train_date_str, "%Y-%m-%d")
config["oot_end_date"] =  config['model_train_date'] - timedelta(days = 0)
config["oot_start_date"] =  config['model_train_date'] - relativedelta(months = oot_period_months)
config["train_test_end_date"] =  config["oot_start_date"] - timedelta(days = 0)
config["train_test_start_date"] =  config["oot_start_date"] - relativedelta(months = train_test_period_months)
config["train_test_ratio"] = train_test_ratio 

pprint.pprint(config)

{'model_train_date': datetime.datetime(2017, 3, 1, 0, 0),
 'model_train_date_str': '2017-03-01',
 'oot_end_date': datetime.datetime(2017, 3, 1, 0, 0),
 'oot_period_months': 0,
 'oot_start_date': datetime.datetime(2017, 3, 1, 0, 0),
 'train_test_end_date': datetime.datetime(2017, 3, 1, 0, 0),
 'train_test_period_months': 0,
 'train_test_ratio': 0.8,
 'train_test_start_date': datetime.datetime(2017, 3, 1, 0, 0)}


## set up config
model_train_date_str = "2017-03-01"
train_test_period_months = 12
oot_period_months = 2
train_test_ratio = 0.8

config = {}
config["model_train_date_str"] = model_train_date_str
config["train_test_period_months"] = train_test_period_months
config["oot_period_months"] =  oot_period_months
config["model_train_date"] =  datetime.strptime(model_train_date_str, "%Y-%m-%d")
config["oot_end_date"] =  config['model_train_date'] - timedelta(days = 1)
config["oot_start_date"] =  config['model_train_date'] - relativedelta(months = oot_period_months)
config["train_test_end_date"] =  config["oot_start_date"] - timedelta(days = 1)
config["train_test_start_date"] =  config["oot_start_date"] - relativedelta(months = train_test_period_months)
config["train_test_ratio"] = train_test_ratio 


pprint.pprint(config)

## Get Label Store

In [4]:
# connect to label store
folder_path = "/app/datamart/gold/label_store_2/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
label_store_sdf = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",label_store_sdf.count())

label_store_sdf.show()

row_count: 7839
+--------------------+-------------+--------+
|                msno|snapshot_date|is_churn|
+--------------------+-------------+--------+
|+3LQldOUM9Zkudhto...|   2017-03-01|       0|
|+5k78SlS0eidCbRK6...|   2017-03-01|       0|
|+79aUW7DYLn4XmBmP...|   2017-03-01|       0|
|+Eemq1EYOUws80NOB...|   2017-03-01|       0|
|+HeHdpWBTTNnBjv1W...|   2017-03-01|       0|
|+KTV2DskKbIH6oxTx...|   2017-03-01|       0|
|+NYKzr17KS0Sw+GFJ...|   2017-03-01|       0|
|+QkCRNYSj14bA+w+C...|   2017-03-01|       1|
|+kqikTzMDrFeBQX0l...|   2017-03-01|       1|
|+ppocLmEmokRaRtxX...|   2017-03-01|       0|
|+uXFPbMT24PGhpOWv...|   2017-03-01|       0|
|+uxvOcxvqcdRIJK77...|   2017-03-01|       0|
|+xLbmcndUaS4o79hG...|   2017-03-01|       0|
|/2i3NIEJ2dmn7KjLl...|   2017-03-01|       0|
|/CSGbq9GzkjjrDoMJ...|   2017-03-01|       0|
|/KVS/6uGX9c6ckvVK...|   2017-03-01|       1|
|/TIClgVl4EJgJrwmp...|   2017-03-01|       0|
|0+NsQGnp0FpHcqjuu...|   2017-03-01|       0|
|00IMysIWMy3LY9cQk

In [5]:
# extract label store
labels_sdf = label_store_sdf.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))

print("extracted labels_sdf", labels_sdf.count(), config["train_test_start_date"], config["oot_end_date"])

extracted labels_sdf 7839 2017-03-01 00:00:00 2017-03-01 00:00:00


## Get Feature Store 

In [6]:
feature_location = "/app/datamart/gold/feature_store/"

In [7]:
# Load CSV into DataFrame - connect to feature store
features_file_list = [feature_location+os.path.basename(f) for f in glob.glob(os.path.join(feature_location, '*'))]

features_store_sdf = spark.read \
                          .option("header", True) \
                          .option("inferSchema", True) \
                          .parquet(*features_file_list)

print("row_count:",features_store_sdf.count())

features_store_sdf.show()

row_count: 37254052
+--------------------+-----------------+-----------------------+--------------+----------+--------------+---------------+------------------+---------------+------------------+------------------+---------------------+--------------+--------------------+-------------------+-----------------------+-----------+----------------+------------------+
|                msno|registration_date|tenure_days_at_snapshot|registered_via|city_clean|        via_oh|        city_oh|      sum_secs_w30|active_days_w30| complete_rate_w30|       sum_secs_w7|engagement_ratio_7_30|last_play_date|days_since_last_play|     trend_secs_w30|latest_transaction_date|tenure_days|auto_renew_share|last_is_auto_renew|
+--------------------+-----------------+-----------------------+--------------+----------+--------------+---------------+------------------+---------------+------------------+------------------+---------------------+--------------+--------------------+-------------------+------------------

In [8]:
features_store_sdf = features_store_sdf.withColumn(
    "snapshot_date",
    F.to_date(F.lit("2017-03-01"), "yyyy-MM-dd")
)

In [9]:
# extract feature store
features_sdf = features_store_sdf.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))

print("extracted features_sdf", features_sdf.count(), config["train_test_start_date"], config["oot_end_date"])

[Stage 14:====================================================>   (48 + 3) / 51]

extracted features_sdf 37254052 2017-03-01 00:00:00 2017-03-01 00:00:00


## Prepare Data for Modelling

In [10]:
# prepare data for modeling
data_pdf = labels_sdf.join(features_sdf, on=["msno", "snapshot_date"], how="left").toPandas()
data_pdf = data_pdf.rename(columns={"is_churn": "label"})
data_pdf

,msno,snapshot_date,label,registration_date,tenure_days_at_snapshot,registered_via,city_clean,via_oh,city_oh,sum_secs_w30,...,complete_rate_w30,sum_secs_w7,engagement_ratio_7_30,last_play_date,days_since_last_play,trend_secs_w30,latest_transaction_date,tenure_days,auto_renew_share,last_is_auto_renew
0,+w+ZTE72WJqfRc4HsKR77d5fhliAZuuXcskgIP3vgUs=,2017-03-01,0,2012-07-09,937.0,3,14,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.000,...,0.000000,0.000,0.000000,None,0.0,0.000000,None,0.0,0.000000,0.0
1,+w+ZTE72WJqfRc4HsKR77d5fhliAZuuXcskgIP3vgUs=,2017-03-01,0,2012-07-09,965.0,3,14,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1715.089,...,0.117647,1388.636,0.809658,2015-02-27,2.0,128.268600,None,0.0,0.000000,0.0
2,+w+ZTE72WJqfRc4HsKR77d5fhliAZuuXcskgIP3vgUs=,2017-03-01,0,2012-07-09,1696.0,3,14,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",45501.522,...,0.671053,14439.054,0.317331,2017-02-28,1.0,16.210799,2017-02-28,1695.0,0.666667,1.0
3,+w+ZTE72WJqfRc4HsKR77d5fhliAZuuXcskgIP3vgUs=,2017-03-01,0,2012-07-09,1696.0,3,14,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",45501.522,...,0.671053,14439.054,0.317331,2017-02-28,1.0,16.210799,2017-02-28,1695.0,0.666667,1.0
4,+w+ZTE72WJqfRc4HsKR77d5fhliAZuuXcskgIP3vgUs=,2017-03-01,0,2012-07-09,1668.0,3,14,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",32309.548,...,0.706667,7710.167,0.238634,2017-02-01,0.0,-89.653589,2017-01-30,1666.0,0.600000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47002,yGsNBN5COIq8McY0uvZ9KholfFYR/wZatWD3BZM9MAI=,2017-03-01,1,2011-01-19,2233.0,7,5,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",13454.697,...,1.388889,0.000,0.000000,2017-02-22,7.0,-713.252429,2017-01-30,2203.0,0.000000,0.0
47003,yGsNBN5COIq8McY0uvZ9KholfFYR/wZatWD3BZM9MAI=,2017-03-01,1,2011-01-19,2205.0,7,5,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1870.848,...,6.000000,1870.848,1.000000,2017-01-30,2.0,0.000000,2017-01-30,2203.0,0.000000,0.0
47004,yGsNBN5COIq8McY0uvZ9KholfFYR/wZatWD3BZM9MAI=,2017-03-01,1,2011-01-19,2174.0,7,5,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000,...,0.000000,0.000,0.000000,None,0.0,0.000000,None,0.0,0.000000,0.0
47005,yGsNBN5COIq8McY0uvZ9KholfFYR/wZatWD3BZM9MAI=,2017-03-01,1,2011-01-19,2143.0,7,5,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000,...,0.000000,0.000,0.000000,None,0.0,0.000000,None,0.0,0.000000,0.0


In [17]:
data_pdf['snapshot_date'] = pd.to_datetime(data_pdf['snapshot_date']).dt.date

In [11]:
features_sdf.columns

['msno',
 'registration_date',
 'tenure_days_at_snapshot',
 'registered_via',
 'city_clean',
 'via_oh',
 'city_oh',
 'sum_secs_w30',
 'active_days_w30',
 'complete_rate_w30',
 'sum_secs_w7',
 'engagement_ratio_7_30',
 'last_play_date',
 'days_since_last_play',
 'trend_secs_w30',
 'latest_transaction_date',
 'tenure_days',
 'auto_renew_share',
 'last_is_auto_renew',
 'snapshot_date']

In [18]:
data_pdf.dtypes

msno                        object
snapshot_date               object
label                        int32
registration_date           object
tenure_days_at_snapshot    float64
registered_via              object
city_clean                  object
via_oh                      object
city_oh                     object
sum_secs_w30               float64
active_days_w30            float64
complete_rate_w30          float64
sum_secs_w7                float64
engagement_ratio_7_30      float64
last_play_date              object
days_since_last_play       float64
trend_secs_w30             float64
latest_transaction_date     object
tenure_days                float64
auto_renew_share           float64
last_is_auto_renew         float64
dtype: object

In [19]:
# split data into train - test - oot
oot_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["oot_start_date"].date()) & (data_pdf['snapshot_date'] <= config["oot_end_date"].date())]
train_test_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["train_test_start_date"].date()) & (data_pdf['snapshot_date'] <= config["train_test_end_date"].date())]

#feature_cols = [fe_col for fe_col in data_pdf.columns if fe_col.startswith('fe_')]
feature_cols = ['tenure_days_at_snapshot',
 'registered_via',
 'city_clean', 
'sum_secs_w30',
 'active_days_w30',
 'complete_rate_w30',
 'sum_secs_w7',
 'engagement_ratio_7_30',
 'days_since_last_play',
 'trend_secs_w30','auto_renew_share',
 'last_is_auto_renew']
                
X_oot = oot_pdf[feature_cols]
y_oot = oot_pdf["label"]
X_train, X_test, y_train, y_test = train_test_split(
    train_test_pdf[feature_cols], train_test_pdf["label"], 
    test_size= 1 - config["train_test_ratio"],
    random_state=88,     # Ensures reproducibility
    shuffle=True,        # Shuffle the data before splitting
    stratify=train_test_pdf["label"]           # Stratify based on the label column
)


print('X_train', X_train.shape[0])
print('X_test', X_test.shape[0])
print('X_oot', X_oot.shape[0])
print('y_train', y_train.shape[0], round(y_train.mean(),2))
print('y_test', y_test.shape[0], round(y_test.mean(),2))
print('y_oot', y_oot.shape[0], round(y_oot.mean(),2))

X_train

X_train 37605
X_test 9402
X_oot 47007
y_train 37605 0.19
y_test 9402 0.19
y_oot 47007 0.19


,tenure_days_at_snapshot,registered_via,city_clean,sum_secs_w30,active_days_w30,complete_rate_w30,sum_secs_w7,engagement_ratio_7_30,days_since_last_play,trend_secs_w30,auto_renew_share,last_is_auto_renew
26754,341.0,4,1,92547.918,15.0,0.898990,20308.780,0.219441,2.0,-186.213907,0.916667,1.0
3276,373.0,4,1,39127.759,21.0,0.679426,10229.058,0.261427,3.0,-8.745568,0.000000,0.0
19519,2260.0,9,5,41308.116,17.0,1.071429,3266.148,0.079068,1.0,-146.378990,1.000000,1.0
45266,1082.0,3,12,573.665,2.0,0.200000,0.000,0.000000,16.0,-197.939000,1.000000,1.0
12733,506.0,9,13,1369.275,2.0,0.555556,0.000,0.000000,11.0,193.375000,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
40309,497.0,4,12,21684.508,11.0,0.468966,2254.493,0.103968,1.0,192.971055,0.000000,0.0
32849,511.0,7,1,6570.659,4.0,0.884615,0.000,0.000000,7.0,-72.278100,1.000000,1.0
6132,3121.0,9,13,47901.902,6.0,0.958333,2963.949,0.061875,1.0,-2680.510514,1.000000,1.0
23123,491.0,9,22,16181.789,9.0,1.774194,131.956,0.008155,0.0,-24.882167,1.000000,1.0


## Preprocess Data

In [20]:
# set up standard scalar preprocessing
scaler = StandardScaler()

transformer_stdscaler = scaler.fit(X_train) 

# transform data
X_train_processed = transformer_stdscaler.transform(X_train)
X_test_processed = transformer_stdscaler.transform(X_test)
X_oot_processed = transformer_stdscaler.transform(X_oot)

print('X_train_processed', X_train_processed.shape[0])
print('X_test_processed', X_test_processed.shape[0])
print('X_oot_processed', X_oot_processed.shape[0])

pd.DataFrame(X_train_processed)

X_train_processed 37605
X_test_processed 9402
X_oot_processed 47007


,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.916083,-0.939599,-1.239397,0.768014,0.515032,-0.027255,0.619243,0.063507,-0.188127,-0.143879,0.717852,0.801102
1,-0.886982,-0.939599,-1.239397,-0.268834,1.287517,-0.253223,-0.105187,0.254182,-0.171290,0.016175,-1.260296,-1.248280
2,0.829079,0.869389,-0.629558,-0.226515,0.772527,0.150212,-0.605612,-0.573976,-0.204964,-0.107953,0.897683,0.801102
3,-0.242209,-1.301396,0.437661,-1.017142,-1.158687,-0.746632,-0.840350,-0.933052,0.047596,-0.154453,0.897683,0.801102
4,-0.766030,0.869389,0.590121,-1.001700,-1.158687,-0.380706,-0.840350,-0.933052,-0.036591,0.198461,0.897683,0.801102
...,...,...,...,...,...,...,...,...,...,...,...,...
37600,-0.774215,-0.939599,0.437661,-0.607395,0.000041,-0.469822,-0.678320,-0.460896,-0.204964,0.198097,-1.260296,-1.248280
37601,-0.761483,0.145794,-1.239397,-0.900744,-0.901192,-0.042049,-0.840350,-0.933052,-0.103940,-0.041123,0.897683,0.801102
37602,1.612082,0.869389,0.590121,-0.098534,-0.643696,0.033819,-0.627331,-0.652053,-0.204964,-2.393412,0.897683,0.801102
37603,-0.779671,0.869389,1.962259,-0.714199,-0.257454,0.873474,-0.830866,-0.896019,-0.221802,0.001622,0.897683,0.801102


## Train Model

#### Logistics Regression

#### XGBoost

In [21]:
# Define the XGBoost classifier
xgb_clf = xgb.XGBClassifier(eval_metric='logloss', random_state=88)

# Define the hyperparameter space to search
param_dist = {
    'n_estimators': [25, 50],
    'max_depth': [2, 3],  # lower max_depth to simplify the model
    'learning_rate': [0.01, 0.1],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 3, 5],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

# Create a scorer based on AUC score
auc_scorer = make_scorer(roc_auc_score)

# Set up the random search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    scoring=auc_scorer,
    n_iter=10,  # Number of iterations for random search
    cv=3,       # Number of folds in cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform the random search
random_search.fit(X_train_processed, y_train)

# Output the best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print("Best AUC score: ", random_search.best_score_)

# Evaluate the model on the train set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_train_processed)[:, 1]
train_auc_score = roc_auc_score(y_train, y_pred_proba)
print("Train AUC score: ", train_auc_score)

# Evaluate the model on the test set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test_processed)[:, 1]
test_auc_score = roc_auc_score(y_test, y_pred_proba)
print("Test AUC score: ", test_auc_score)

# Evaluate the model on the oot set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("TRAIN GINI score: ", round(2*train_auc_score-1,3))
print("Test GINI score: ", round(2*test_auc_score-1,3))
print("OOT GINI score: ", round(2*oot_auc_score-1,3))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters found:  {'subsample': 0.6, 'reg_lambda': 1.5, 'reg_alpha': 1, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best AUC score:  0.6178269515156666
Train AUC score:  0.806623816380188
Test AUC score:  0.7994591535477367
OOT AUC score:  0.8052332510959205
TRAIN GINI score:  0.613
Test GINI score:  0.599
OOT GINI score:  0.61


## Prepare model artefact to save

In [22]:
model_artefact = {}

model_artefact['model'] = best_model
model_artefact['model_version'] = "credit_model_label2" + config['model_train_date_str'].replace('-','_')
model_artefact['preprocessing_transformers'] = {}
model_artefact['preprocessing_transformers']['stdscaler'] = transformer_stdscaler
model_artefact['data_dates'] = config
model_artefact['data_stats'] = {}
model_artefact['data_stats']['X_train'] = X_train.shape[0]
model_artefact['data_stats']['X_test'] = X_test.shape[0]
model_artefact['data_stats']['X_oot'] = X_oot.shape[0]
model_artefact['data_stats']['y_train'] = round(y_train.mean(),2)
model_artefact['data_stats']['y_test'] = round(y_test.mean(),2)
model_artefact['data_stats']['y_oot'] = round(y_oot.mean(),2)
model_artefact['results'] = {}
model_artefact['results']['auc_train'] = train_auc_score
model_artefact['results']['auc_test'] = test_auc_score
model_artefact['results']['auc_oot'] = oot_auc_score
model_artefact['results']['gini_train'] = round(2*train_auc_score-1,3)
model_artefact['results']['gini_test'] = round(2*test_auc_score-1,3)
model_artefact['results']['gini_oot'] = round(2*oot_auc_score-1,3)
model_artefact['hp_params'] = random_search.best_params_


pprint.pprint(model_artefact)

{'data_dates': {'model_train_date': datetime.datetime(2017, 3, 1, 0, 0),
                'model_train_date_str': '2017-03-01',
                'oot_end_date': datetime.datetime(2017, 3, 1, 0, 0),
                'oot_period_months': 0,
                'oot_start_date': datetime.datetime(2017, 3, 1, 0, 0),
                'train_test_end_date': datetime.datetime(2017, 3, 1, 0, 0),
                'train_test_period_months': 0,
                'train_test_ratio': 0.8,
                'train_test_start_date': datetime.datetime(2017, 3, 1, 0, 0)},
 'data_stats': {'X_oot': 47007,
                'X_test': 9402,
                'X_train': 37605,
                'y_oot': np.float64(0.19),
                'y_test': np.float64(0.19),
                'y_train': np.float64(0.19)},
 'hp_params': {'colsample_bytree': 0.8,
               'gamma': 0.1,
               'learning_rate': 0.1,
               'max_depth': 3,
               'min_child_weight': 5,
               'n_estimators': 50,
         

## Save Artefact to Model Bank

In [23]:
# create model_bank dir
model_bank_directory = "model_bank/"

if not os.path.exists(model_bank_directory):
    os.makedirs(model_bank_directory)

In [24]:
# Full path to the file
file_path = os.path.join(model_bank_directory, model_artefact['model_version'] + '.pkl')

# Write the model to a pickle file
with open(file_path, 'wb') as file:
    pickle.dump(model_artefact, file)

print(f"Model saved to {file_path}")


Model saved to model_bank/credit_model_label22017_03_01.pkl


## Test Load Pickle and Make Model Inference

In [25]:
# Load the model from the pickle file
with open(file_path, 'rb') as file:
    loaded_model_artefact = pickle.load(file)

y_pred_proba = loaded_model_artefact['model'].predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("Model loaded successfully!")

OOT AUC score:  0.8052332510959205
Model loaded successfully!


In [26]:
spark.stop()